# Sacs à dos multiple (Spécial)

Objectif : Grouper des objets de la même longueur dans le moins de sacs possibles

Lien : https://stackoverflow.com/questions/73111293/how-to-apply-a-binary-constraint-on-a-binary-variable-in-pulp

In [3]:
import pulp
from itertools import product
import pandas as pd

# DataFrame of item, weight, and length
df_updated = pd.DataFrame([['item1', 10, 'A'], ['item2', 15, 'A'], ['item3',15, 'B'], ['item4',15, 'C']], columns = ['itemname', 'weight', 'length'])

# Max bin to use
max_bins = 2

# Max weightage per bin
max_weight = 30

problem = pulp.LpProblem("Grouping_lengths", pulp.LpMinimize)

# Variable to check, if we are using the bin or not
bin_used = pulp.LpVariable.dicts('is_bin_used', range(max_bins), cat='Binary')

# Possible combinations to put the item in the bin
possible_item_in_bin = [(item_index, bin_num) for item_index, bin_num in product(df_updated.index, range(max_bins))]
item_in_bin = pulp.LpVariable.dicts('is_item_in_bin', possible_item_in_bin, cat = 'Binary')

# Only one item in each bin
for item_index in df_updated.index:
    problem += pulp.lpSum([item_in_bin[item_index, bin_index] for bin_index in range(max_bins)]) == 1, f"Ensure that item {item_index} is only in one bin"

# Sum of quantity grouped in each bin must be less than max weight
for bin_index in range(max_bins):
    problem += pulp.lpSum(
            [item_in_bin[item_index, bin_index] * df_updated.loc[item_index, 'weight'] for item_index in df_updated.index]
        ) <= max_weight* bin_used[bin_index], f"Sum of items in bin {bin_index} should not exceed max weight {max_weight}"


 # Length Constraints
lengths = list(df_updated.length.unique())
for length in lengths:
    items_n = df_updated.index[df_updated['length'] == length].tolist()  # get items with given length
    if len(items_n) > 1:  # skip items with unique length
        for bin in range(max_bins - 1):  # for each bin except the last one because the last can be deduced
            for item in range(1, len(items_n)):  # set other item assignment equal to the first one.
                problem += item_in_bin[0, bin] == item_in_bin[item, bin]

# Objective function to minimize bins used
problem += pulp.lpSum(bin_used[bin_index] for bin_index in range(max_bins)), "Objective: Minimize Bins Used"

problem.solve(pulp.PULP_CBC_CMD(msg = False))

for val in problem.variables():
    if val.varValue == 1:
        print(val.name, val.varValue)

is_bin_used_0 1.0
is_bin_used_1 1.0
is_item_in_bin_(0,_1) 1.0
is_item_in_bin_(1,_1) 1.0
is_item_in_bin_(2,_0) 1.0
is_item_in_bin_(3,_0) 1.0
